# 🚀 股票预测系统 - Google Colab 版本

这个notebook演示了如何在Google Colab中运行股票预测系统，使用本地LLM进行预测分析。

## 🎯 功能特性
- 实时股票数据获取
- 技术指标计算
- AI驱动的股票预测
- 本地LLM分析（无需API密钥）
- 交互式Web界面

## 📋 使用步骤
1. 运行第一个cell安装依赖
2. 运行第二个cell启动服务
3. 访问Web界面进行股票预测


## 🔧 步骤1: 安装依赖和设置环境


In [ ]:
# 安装系统依赖
!apt-get update
!apt-get install -y curl wget

# 安装Ollama
!curl -fsSL https://ollama.com/install.sh | sh

print("✅ 系统依赖安装完成")


In [ ]:
# 安装Python依赖
!pip install -q fastapi uvicorn langgraph langchain langchain-openai yfinance pandas numpy ta pydantic python-multipart python-dotenv httpx aiofiles pyarrow alpha-vantage requests aiohttp ollama jupyter-dash plotly dash

print("✅ Python依赖安装完成")


In [ ]:
# 启动Ollama服务
import subprocess
import time
import threading

def start_ollama():
    subprocess.run(['ollama', 'serve'], check=True)

# 在后台启动Ollama
ollama_thread = threading.Thread(target=start_ollama, daemon=True)
ollama_thread.start()

# 等待服务启动
time.sleep(10)
print("✅ Ollama服务已启动")


In [ ]:
# 下载LLM模型
!ollama pull qwen2.5:7b

# 验证模型
!ollama list

print("✅ 模型下载完成")


In [ ]:
# 设置环境变量
import os

os.environ["LLM_TYPE"] = "ollama"
os.environ["OLLAMA_MODEL"] = "qwen2.5:7b"
os.environ["HOST"] = "0.0.0.0"
os.environ["PORT"] = "8000"
os.environ["CACHE_TTL_HOURS"] = "24"

print("✅ 环境变量已配置")


In [ ]:
# 测试Ollama连接
import ollama

try:
    response = ollama.chat(
        model="qwen2.5:7b",
        messages=[{'role': 'user', 'content': 'Hello, are you working?'}]
    )
    print("✅ Ollama测试成功")
    print(f"回复: {response['message']['content']}")
except Exception as e:
    print(f"❌ Ollama测试失败: {e}")


## 📁 步骤2: 上传项目文件

**重要**: 在启动应用之前，必须先上传项目文件！

### 方法1: 手动上传（推荐）
1. 点击左侧的文件夹图标 📁
2. 点击"上传到会话存储"按钮
3. 选择并上传整个项目文件夹
4. 确保 `backend/` 文件夹在根目录下

### 方法2: 从GitHub克隆
如果你已经将项目推送到GitHub，可以使用以下命令：


In [ ]:
# 从GitHub克隆项目
!git clone https://github.com/shilinzmax/stockprediction.git
%cd stockprediction/stock-predictor

# 检查项目文件是否存在
import os
if os.path.exists('backend'):
    print("✅ 找到backend文件夹")
    print("📁 项目文件已就绪")
    print("🎉 成功从GitHub克隆项目！")
else:
    print("❌ 未找到backend文件夹")
    print("📁 请检查GitHub仓库结构")
    print("💡 或者使用左侧文件面板手动上传项目文件夹")


In [ ]:
# 检查项目文件是否正确克隆
import os

def check_project_files():
    """检查项目文件是否存在"""
    print("🔍 检查项目文件...")
    
    required_items = [
        'backend/',
        'backend/app.py',
        'backend/core/',
        'backend/core/llm.py',
        'backend/core/state.py',
        'backend/graph/',
        'backend/graph/pipeline.py'
    ]
    
    missing_items = []
    existing_items = []
    
    for item in required_items:
        if os.path.exists(item):
            existing_items.append(item)
            print(f"✅ {item}")
        else:
            missing_items.append(item)
            print(f"❌ {item}")
    
    print(f"\n📊 检查结果:")
    print(f"✅ 存在: {len(existing_items)} 个文件/文件夹")
    print(f"❌ 缺失: {len(missing_items)} 个文件/文件夹")
    
    if missing_items:
        print(f"\n❌ 缺少以下文件/文件夹:")
        for item in missing_items:
            print(f"  - {item}")
        
        print(f"\n💡 解决方案:")
        print("1. 运行上面的GitHub克隆cell")
        print("2. 或者使用左侧文件面板上传项目文件夹")
        print("3. 确保项目结构正确")
        return False
    else:
        print(f"\n🎉 所有文件都已正确克隆！")
        print("📁 项目文件已就绪，可以启动应用了")
        return True

# 运行检查
files_ok = check_project_files()


## 🚀 步骤3: 启动应用


In [ ]:
# 启动FastAPI应用
import uvicorn
import os
import sys

# 切换到正确的目录
os.chdir('/content/stockprediction/stock-predictor')

# 添加当前目录到Python路径
sys.path.append('/content/stockprediction/stock-predictor')
sys.path.append('/content/stockprediction')

# 检查项目文件是否存在
if not os.path.exists('backend'):
    print("❌ 错误: 未找到backend文件夹")
    print("📁 请先运行上面的GitHub克隆cell")
    print("💡 或者使用左侧文件面板上传项目文件夹")
    print("🔄 文件准备完成后重新运行此cell")
else:
    try:
        # 尝试导入应用
        from backend.app import app
        
        print("🚀 启动股票预测应用...")
        print("📱 应用将在 http://localhost:8000 启动")
        print("📊 API文档: http://localhost:8000/docs")
        print("🔗 健康检查: http://localhost:8000/health")
        print("📈 预测API: POST http://localhost:8000/predict")
        print("\n按 Ctrl+C 停止应用")
        
        # 启动服务器
        uvicorn.run(app, host="0.0.0.0", port=8000)
        
    except ImportError as e:
        print(f"❌ 导入错误: {e}")
        print("📁 请确保backend文件夹包含所有必要的Python文件")
        print("💡 检查backend/app.py文件是否存在")
        print("🔄 尝试重新克隆项目或上传文件")
    except Exception as e:
        print(f"❌ 启动失败: {e}")
        print("🔄 请检查错误信息并重试")
        print("💡 确保所有依赖都已正确安装")


## 🔧 快速解决方案（不修改原始文件）

### 问题1: "ModuleNotFoundError: No module named 'backend'"

**原因**: Python路径问题，无法找到backend模块

**解决方案**:
1. 运行"方法3: 使用命令行启动"cell（最简单）
2. 或者运行"方法4: 动态导入修复"cell
3. 确保在正确的目录下运行

### 问题2: "ModuleNotFoundError: No module named 'core'"

**原因**: 导入路径问题，无法找到core模块

**解决方案**:
1. 运行"方法4: 动态导入修复"cell（推荐）
2. 或者运行"方法1: 创建符号链接"cell
3. 或者使用命令行启动方法

### 问题3: 项目文件缺失

**解决方案**:
1. 运行GitHub克隆cell
2. 运行文件检查cell验证结构
3. 确保所有文件都已正确下载

### 推荐启动顺序（不修改文件）:
1. 运行GitHub克隆cell
2. 运行文件检查cell
3. 运行"方法3: 使用命令行启动"cell
4. 如果失败，运行"方法4: 动态导入修复"cell
5. 最后尝试"方法1: 创建符号链接"cell


In [ ]:
# 解决路径问题的启动方法
import os
import sys

# 切换到正确的目录
os.chdir('/content/stockprediction/stock-predictor')

# 添加Python路径
sys.path.append('/content/stockprediction/stock-predictor')

# 检查当前目录和文件
print(f"当前目录: {os.getcwd()}")
print(f"Python路径: {sys.path[:3]}...")
print(f"backend文件夹存在: {os.path.exists('backend')}")

# 启动应用
try:
    import uvicorn
    from backend.app import app
    
    print("🚀 启动股票预测应用...")
    print("📱 应用将在 http://localhost:8000 启动")
    print("📊 API文档: http://localhost:8000/docs")
    print("🔗 健康检查: http://localhost:8000/health")
    print("📈 预测API: POST http://localhost:8000/predict")
    print("\n按 Ctrl+C 停止应用")
    
    uvicorn.run(app, host="0.0.0.0", port=8000)
    
except ImportError as e:
    print(f"❌ 导入错误: {e}")
    print("💡 尝试使用命令行启动:")
    print("!cd /content/stockprediction/stock-predictor && python -m uvicorn backend.app:app --host 0.0.0.0 --port 8000")
except Exception as e:
    print(f"❌ 启动失败: {e}")
    print("🔄 请检查错误信息并重试")


In [ ]:
# 命令行启动方法（如果上面的方法失败）
!cd /content/stockprediction/stock-predictor && python -m uvicorn backend.app:app --host 0.0.0.0 --port 8000


In [ ]:
# 修复导入问题后的启动方法
import os
import sys

# 切换到正确的目录
os.chdir('/content/stockprediction/stock-predictor')

# 添加Python路径
sys.path.append('/content/stockprediction/stock-predictor')
sys.path.append('/content/stockprediction')

# 检查文件结构
print(f"当前目录: {os.getcwd()}")
print(f"backend存在: {os.path.exists('backend')}")
print(f"backend/app.py存在: {os.path.exists('backend/app.py')}")

# 尝试启动应用
try:
    import uvicorn
    from backend.app import app
    
    print("🚀 启动股票预测应用...")
    print("📱 应用将在 http://localhost:8000 启动")
    print("📊 API文档: http://localhost:8000/docs")
    print("🔗 健康检查: http://localhost:8000/health")
    print("📈 预测API: POST http://localhost:8000/predict")
    print("\n按 Ctrl+C 停止应用")
    
    uvicorn.run(app, host="0.0.0.0", port=8000)
    
except ImportError as e:
    print(f"❌ 导入错误: {e}")
    print("💡 导入问题已修复，请重新运行此cell")
except Exception as e:
    print(f"❌ 启动失败: {e}")
    print("🔄 请检查错误信息并重试")


In [ ]:
# 修复导入问题
import os
import re

def fix_imports_in_file(file_path):
    """修复单个文件中的导入"""
    print(f"🔧 修复文件: {file_path}")
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # 修复core导入
        content = re.sub(r'^from core\.', 'from backend.core.', content, flags=re.MULTILINE)
        content = re.sub(r'^from graph\.', 'from backend.graph.', content, flags=re.MULTILINE)
        
        # 写回文件
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        
        print(f"✅ 修复完成: {file_path}")
        return True
        
    except Exception as e:
        print(f"❌ 修复失败: {file_path} - {e}")
        return False

# 切换到项目目录
os.chdir('/content/stockprediction/stock-predictor')

# 需要修复的文件列表
files_to_fix = [
    'backend/test_ollama.py',
    'backend/test_gpt_api.py',
    'backend/graph/nodes/llm_analyze.py',
    'backend/graph/nodes/fetch_data.py',
    'backend/graph/pipeline.py',
    'backend/graph/nodes/report.py',
    'backend/graph/nodes/make_advice.py',
    'backend/graph/nodes/feature_engineer.py'
]

success_count = 0
for file_path in files_to_fix:
    if os.path.exists(file_path):
        if fix_imports_in_file(file_path):
            success_count += 1
    else:
        print(f"⚠️ 文件不存在: {file_path}")

print(f"\n🎉 修复完成！成功修复: {success_count}/{len(files_to_fix)} 个文件")

# 测试导入
print("\n🧪 测试导入...")
try:
    from backend.app import app
    print("✅ 应用导入成功！")
    print("🚀 现在可以启动应用了！")
except Exception as e:
    print(f"❌ 应用导入失败: {e}")
    print("🔄 请检查错误信息并重试")


In [ ]:
# 不修改原始文件的Colab启动方法
import os
import sys

# 切换到正确的目录
os.chdir('/content/stockprediction/stock-predictor')

# 添加Python路径
sys.path.append('/content/stockprediction/stock-predictor')
sys.path.append('/content/stockprediction')

# 创建符号链接来解决导入问题
def create_symlinks():
    """创建符号链接来解决导入问题"""
    print("🔗 创建符号链接...")
    
    try:
        # 在backend目录下创建core和graph的符号链接
        if not os.path.exists('/content/stockprediction/stock-predictor/backend/core'):
            os.symlink('/content/stockprediction/stock-predictor/backend/core', 
                      '/content/stockprediction/stock-predictor/core')
            print("✅ 创建core符号链接")
        
        if not os.path.exists('/content/stockprediction/stock-predictor/backend/graph'):
            os.symlink('/content/stockprediction/stock-predictor/backend/graph', 
                      '/content/stockprediction/stock-predictor/graph')
            print("✅ 创建graph符号链接")
        
        return True
    except Exception as e:
        print(f"❌ 创建符号链接失败: {e}")
        return False

# 创建符号链接
create_symlinks()

# 检查文件结构
print(f"\n📁 当前目录: {os.getcwd()}")
print(f"📁 backend存在: {os.path.exists('backend')}")
print(f"📁 core符号链接存在: {os.path.exists('core')}")
print(f"📁 graph符号链接存在: {os.path.exists('graph')}")

# 尝试启动应用
try:
    import uvicorn
    from backend.app import app
    
    print("\n🚀 启动股票预测应用...")
    print("📱 应用将在 http://localhost:8000 启动")
    print("📊 API文档: http://localhost:8000/docs")
    print("🔗 健康检查: http://localhost:8000/health")
    print("📈 预测API: POST http://localhost:8000/predict")
    print("\n按 Ctrl+C 停止应用")
    
    uvicorn.run(app, host="0.0.0.0", port=8000)
    
except ImportError as e:
    print(f"\n❌ 导入错误: {e}")
    print("💡 尝试其他方法...")
except Exception as e:
    print(f"\n❌ 启动失败: {e}")
    print("🔄 请检查错误信息并重试")


In [ ]:
# 方法2: 使用PYTHONPATH环境变量
import os
import sys

# 设置PYTHONPATH环境变量
os.environ['PYTHONPATH'] = '/content/stockprediction/stock-predictor'

# 切换到正确的目录
os.chdir('/content/stockprediction/stock-predictor')

# 添加Python路径
sys.path.insert(0, '/content/stockprediction/stock-predictor')

print(f"📁 当前目录: {os.getcwd()}")
print(f"🐍 PYTHONPATH: {os.environ.get('PYTHONPATH')}")
print(f"📁 backend存在: {os.path.exists('backend')}")

# 尝试启动应用
try:
    import uvicorn
    from backend.app import app
    
    print("\n🚀 启动股票预测应用...")
    print("📱 应用将在 http://localhost:8000 启动")
    print("📊 API文档: http://localhost:8000/docs")
    print("🔗 健康检查: http://localhost:8000/health")
    print("📈 预测API: POST http://localhost:8000/predict")
    print("\n按 Ctrl+C 停止应用")
    
    uvicorn.run(app, host="0.0.0.0", port=8000)
    
except ImportError as e:
    print(f"\n❌ 导入错误: {e}")
    print("💡 尝试方法3...")
except Exception as e:
    print(f"\n❌ 启动失败: {e}")
    print("🔄 请检查错误信息并重试")


In [ ]:
# 方法3: 使用命令行启动（最简单）
import os

# 切换到正确的目录
os.chdir('/content/stockprediction/stock-predictor')

print("🚀 使用命令行启动应用...")
print("📱 应用将在 http://localhost:8000 启动")
print("📊 API文档: http://localhost:8000/docs")
print("🔗 健康检查: http://localhost:8000/health")
print("📈 预测API: POST http://localhost:8000/predict")
print("\n按 Ctrl+C 停止应用")

# 使用命令行启动
!python -m uvicorn backend.app:app --host 0.0.0.0 --port 8000


In [ ]:
# 方法4: 动态导入修复（不修改文件）
import os
import sys
import importlib.util

# 切换到正确的目录
os.chdir('/content/stockprediction/stock-predictor')

# 添加Python路径
sys.path.insert(0, '/content/stockprediction/stock-predictor')

def fix_imports_dynamically():
    """动态修复导入问题"""
    print("🔧 动态修复导入问题...")
    
    # 创建core模块的别名
    import backend.core as core
    sys.modules['core'] = core
    
    # 创建graph模块的别名
    import backend.graph as graph
    sys.modules['graph'] = graph
    
    print("✅ 动态导入修复完成")

# 执行动态修复
fix_imports_dynamically()

# 尝试启动应用
try:
    import uvicorn
    from backend.app import app
    
    print("\n🚀 启动股票预测应用...")
    print("📱 应用将在 http://localhost:8000 启动")
    print("📊 API文档: http://localhost:8000/docs")
    print("🔗 健康检查: http://localhost:8000/health")
    print("📈 预测API: POST http://localhost:8000/predict")
    print("\n按 Ctrl+C 停止应用")
    
    uvicorn.run(app, host="0.0.0.0", port=8000)
    
except ImportError as e:
    print(f"\n❌ 导入错误: {e}")
    print("💡 请尝试其他方法")
except Exception as e:
    print(f"\n❌ 启动失败: {e}")
    print("🔄 请检查错误信息并重试")


## 🧪 步骤4: 测试API


In [ ]:
# 测试API端点
import requests
import json

# 测试健康检查
try:
    response = requests.get("http://localhost:8000/health")
    print("✅ 健康检查:", response.json())
except Exception as e:
    print(f"❌ 健康检查失败: {e}")


In [ ]:
# 测试股票预测
try:
    prediction_data = {
        "symbol": "AAPL",
        "timeframe": "1d"
    }
    
    response = requests.post(
        "http://localhost:8000/predict",
        json=prediction_data
    )
    
    if response.status_code == 200:
        result = response.json()
        print("✅ 股票预测成功:")
        print(json.dumps(result, indent=2, ensure_ascii=False))
    else:
        print(f"❌ 预测失败: {response.status_code}")
        print(response.text)
        
except Exception as e:
    print(f"❌ 预测请求失败: {e}")


## 📝 注意事项

1. **会话时间限制**: Colab会话有12小时限制
2. **内存使用**: 监控内存使用情况，避免超出限制
3. **数据持久化**: 重要数据请保存到Google Drive
4. **模型大小**: Qwen2.5:7b模型约4.7GB，确保有足够空间
5. **网络访问**: 确保可以访问外部API获取股票数据

## 🎉 完成！

现在你可以在Colab中运行股票预测系统了！

### 访问方式：
- **API文档**: http://localhost:8000/docs
- **健康检查**: http://localhost:8000/health
- **股票预测**: POST http://localhost:8000/predict

### 使用示例：
```python
import requests

# 预测AAPL股票
response = requests.post("http://localhost:8000/predict", 
                        json={"symbol": "AAPL", "timeframe": "1d"})
result = response.json()
print(result)
```
